# FalkorDBQAChain

This notebook shows how to use LLMs to provide a natural language interface to FalkorDB database.

FalkorDB is a low latency property graph database management system. You can simply run its docker locally:

```bash
docker run -p 6379:6379 -it --rm falkordb/falkordb:edge
```

Once launched, you can simply start creating a database on the local machine and connect to it.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.graphs import FalkorDBGraph
from langchain.chains import FalkorDBQAChain

In [2]:
graph = FalkorDBGraph(database="movies")

In [3]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

In [4]:
graph.refresh_schema()
import os
os.environ['OPENAI_API_KEY']='API_KEY_HERE'


In [5]:
chain = FalkorDBQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [6]:
chain.run("Who played in Top Gun?")



> Entering new FalkorDBQAChain chain...
Generated Cypher:
MATCH (:Movie {title: 'Top Gun'})<-[:ACTED_IN]-(actor:Person)
RETURN actor.name AS output
Full Context:
[]

> Finished chain.


'The actor who played in Top Gun is Tom Cruise.'